In [1]:
# -*- coding: utf-8 -*-
# License: BSD
# Author: Sasank Chilamkurthy

from __future__ import print_function, division

import torch
import torch.nn as nn
from torch import optim, cuda
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torchsummary import summary
import matplotlib.pyplot as plt
import time
import os
import copy
import pandas as pd
import cv2 as cv
from PIL import Image
import load_data
from load_data import ForamDataSet
from importlib import reload
from timeit import default_timer as timer

In [11]:
train_on_gpu = torch.cuda.is_available()
save_file_name = 'vgg16-transfer-4.pt'
checkpoint_path = 'vgg16-transfer-4.pth'

# Whether to train on a gpu
train_on_gpu = cuda.is_available()
print('Train on gpu: {train_on_gpu}'.format(train_on_gpu=train_on_gpu))
multi_gpu = False
# Number of gpus
if train_on_gpu:
    gpu_count = cuda.device_count()
    print('{gpu_count} gpus detected.'.format(gpu_count=gpu_count))
    if gpu_count > 1:
        multi_gpu = True

Train on gpu: False


In [3]:
data_transforms = {
    'train': transforms.Compose([
        transforms.ToPILImage(),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    # val does not use augmentation
    'val': transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    # test does not use augmentation
    'test': transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [4]:
data_dir = '../media'
image_datasets = {}
image_datasets['train'] = ForamDataSet(csv_file='../train.csv',
                                       root_dir='../media',
                                       transform=data_transforms['train'])
image_datasets['val'] = ForamDataSet(csv_file='../val.csv',
                                     root_dir='../media',
                                     transform=data_transforms['val'])
image_datasets['test'] = ForamDataSet(csv_file='../test.csv',
                                     root_dir='../media',
                                     transform=data_transforms['test'])                                     
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val', 'test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val', 'test']}
classes = image_datasets['train'].labels

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [7]:
def load_checkpoint(path):
    """Load a PyTorch model checkpoint

    Params
    --------
        path (str): saved model checkpoint. Must start with `model_name-` and end in '.pth'

    Returns
    --------
        None, save the `model` to `path`

    """

    # Get the model name
    model_name = path.split('-')[0]

    # Load in checkpoint
    checkpoint = torch.load(path, map_location='cpu')

    if model_name == 'vgg16':
        model = models.vgg16(pretrained=True)
        # Make sure to set parameters as not trainable
        for param in model.parameters():
            param.requires_grad = False
        model.classifier = checkpoint['classifier']

    elif model_name == 'resnet50':
        model = models.resnet50(pretrained=True)
        # Make sure to set parameters as not trainable
        for param in model.parameters():
            param.requires_grad = False
        model.fc = checkpoint['fc']

    # Load in the state dict
    model.load_state_dict(checkpoint['state_dict'])

    total_params = sum(p.numel() for p in model.parameters())

    # Move to gpu
    if multi_gpu:
        model = nn.DataParallel(model)

    if train_on_gpu:
        model = model.to('cuda')

    # Model basics
    model.idx_to_class = checkpoint['idx_to_class']
    model.epochs = checkpoint['epochs']

    # Optimizer
    optimizer = checkpoint['optimizer']
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

    return model, optimizer

In [12]:
model, optimizer = load_checkpoint('vgg16-transfer-4.pth')

In [14]:
summary(model, input_size=(3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
             ReLU-12          [-1, 256, 56, 56]               0
           Conv2d-13          [-1, 256, 56, 56]         590,080
             ReLU-14          [-1, 256,

In [17]:
def accuracy(output, target, topk=(1, )):
    """
    Compute the topk accuracy(s)
    target: the correct answer
    """
    if train_on_gpu:
        output = output.to('cuda')
        target = target.to('cuda')

    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        # Find the predicted classes and transpose
        _, pred = output.topk(k=maxk, dim=1, largest=True, sorted=True)
        pred = pred.t()
        # Determine predictions equal to the targets
        correct = pred.eq(target.view(1, -1).expand_as(pred))
        res = []

        # For each k, find the percentage of correct
        for k in topk:
            correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size).item())
        return res

In [18]:
def evaluate_for_categories(model, test_loader, criterion, topk=(1, 5)):
    """Measure the performance of a trained PyTorch model

    Params
    --------
        model (PyTorch model): trained cnn for inference
        test_loader (PyTorch DataLoader): test dataloader
        topk (tuple of ints): accuracy to measure

    Returns
    --------
        results (DataFrame): results for each category

    """

    classes = []
    losses = []
    # Hold accuracy results
    acc_results = np.zeros((len(test_loader.dataset), len(topk)))
    i = 0

    model.eval()
    with torch.no_grad():

        # Testing loop
        for data, targets in test_loader:

            # Tensors to gpu
            if train_on_gpu:
                data, targets = data.to('cuda'), targets.to('cuda')

            # Raw model output
            out = model(data)
            # Iterate through each example
            for pred, true in zip(out, targets):
                # Find topk accuracy
                acc_results[i, :] = accuracy(
                    pred.unsqueeze(0), true.unsqueeze(0), topk)
                classes.append(model.idx_to_class[true.item()])
                # Calculate the loss
                loss = criterion(pred.view(1, 17), true.view(1))
                losses.append(loss.item())
                i += 1

    # Send results to a dataframe and calculate average across classes
    for i in topk:
        results = pd.DataFrame(acc_results, columns=['top{i}'.format(i=i) for i in topk])
        results['class'] = classes
        results['loss'] = losses
        results = results.groupby(classes).mean()

    return results.reset_index().rename(columns={'index': 'class'})

In [19]:
criterion = nn.CrossEntropyLoss()
results = evaluate_for_categories(model, dataloaders['test'], criterion)

In [21]:
print(results)

                  class       top1        top5      loss
0       G. crassaformis   0.000000   69.230769  2.975598
1          G. elongatus  42.857143   95.238095  1.294481
2              G. ruber  80.000000   94.285714  0.929556
3         G. ruber pink   0.000000  100.000000  2.023419
4         G. sacculifer  92.105263  100.000000  0.425107
5            G. scitula  53.333333   86.666667  1.645150
6        G. siphonifera  74.285714   91.428571  0.942716
7   G. truncatulinoides  79.069767   93.023256  0.969238
8             G. tumida  80.000000   97.142857  0.657629
9           G. ungulata  78.947368   94.736842  0.812657
10       N. acostaensis  52.380952   90.476190  1.907194
11         N. dutertrei  78.571429  100.000000  0.747720
12          N. humerosa  64.000000   88.000000  1.599275
13          O. universa  60.000000  100.000000  0.408932
14   P. obliquiloculata  54.285714   97.142857  1.096761
15          S. dehiscen  80.555556   97.222222  0.749842


In [33]:
def F1(model, test_loader):
    '''
    class_info = {class_index: [true_postiive,
                                false_positive,
                                total_per_species,
                                precision,
                                recall,
                                F-score]}
    '''
    model.eval()
    class_info = {}
    for i in range(len(image_datasets['train'].labels)):
        class_info[i] = [0,0,0]
    with torch.no_grad():
        for data, targets in test_loader:
            # Tensors to gpu
            if train_on_gpu:
                data, targets = data.to('cuda'), targets.to('cuda')
                # Raw model output
            out = model(data)
            _, pred = out.topk(k=1, dim=1, largest=True, sorted=True)
            pred = torch.squeeze(pred, 1)
            for i in range(len(targets)):
                if pred[i] == targets[i]: # prediction correct, add to true positive
                    temp = class_info[int(pred[i])]
                    temp[0] = temp[0] + 1
                    class_info[int(pred[i])] = temp
                else: # prediction wrong, add to false positive
                    temp = class_info[int(pred[i])]
                    temp[1] = temp[1] + 1
                    class_info[int(pred[i])] = temp
    test_csv = pd.read_csv('../test.csv').groupby('species').count()
    species = image_datasets['train'].labels
    print(class_info)
    for i in range(test_csv.shape[0]):
        row = test_csv.iloc[i]
        print(row)
        arr = class_info[species.index(row.name)]
        if arr == [0,0,0]:
            continue
        print(arr)
        arr[2] = int(row.location)
        arr.append(arr[0]/(arr[0]+arr[1]))
        arr.append(arr[0]/arr[2])
        try:
            arr.append((2*arr[3]*arr[4])/(arr[3]+arr[4]))
        except ZeroDivisionError:
            arr.append(float('nan'))
        class_info[species.index(row.name)] = arr
    total_true_positives = 0
    total_false_postives = 0
    total = 0
    for key,values in class_info.items():
        total_true_positives += values[0]
        total_false_postives += values[1]
        total += values[2]
    micro_avg_prec = total_true_positives/(total_true_positives+total_false_postives)
    micro_avg_rec = total_true_positives/total
    micro_f1 = (2*micro_avg_prec*micro_avg_rec)/(micro_avg_prec+micro_avg_rec)
    df = pd.DataFrame.from_dict(class_info, orient='index',
                        columns=['true_positive','false_positive','total_per_species','precision','recall','F-score'])
    return micro_f1, df

In [34]:
micro_f1, data_stuff = F1(model, dataloaders['test'])

{0: [0, 1, 0], 1: [9, 1, 0], 2: [0, 0, 0], 3: [28, 14, 0], 4: [0, 0, 0], 5: [35, 22, 0], 6: [16, 4, 0], 7: [26, 5, 0], 8: [34, 44, 0], 9: [28, 1, 0], 10: [15, 4, 0], 11: [11, 5, 0], 12: [11, 13, 0], 13: [16, 6, 0], 14: [3, 3, 0], 15: [19, 9, 0], 16: [29, 9, 0]}
location    26
Name: G. crassaformis, dtype: int64
[0, 1, 0]
location    21
Name: G. elongatus, dtype: int64
[9, 1, 0]
location    35
Name: G. ruber, dtype: int64
[28, 14, 0]
location    3
Name: G. ruber pink, dtype: int64
location    38
Name: G. sacculifer, dtype: int64
[35, 22, 0]
location    30
Name: G. scitula, dtype: int64
[16, 4, 0]
location    35
Name: G. siphonifera, dtype: int64
[26, 5, 0]
location    43
Name: G. truncatulinoides, dtype: int64
[34, 44, 0]
location    35
Name: G. tumida, dtype: int64
[28, 1, 0]
location    19
Name: G. ungulata, dtype: int64
[15, 4, 0]
location    21
Name: N. acostaensis, dtype: int64
[11, 5, 0]
location    14
Name: N. dutertrei, dtype: int64
[11, 13, 0]
location    25
Name: N. humerosa, 

In [36]:
print(data_stuff)

    true_positive  false_positive  total_per_species  precision    recall  \
0               0               1                 26   0.000000  0.000000   
1               9               1                 21   0.900000  0.428571   
2               0               0                  0        NaN       NaN   
3              28              14                 35   0.666667  0.800000   
4               0               0                  0        NaN       NaN   
5              35              22                 38   0.614035  0.921053   
6              16               4                 30   0.800000  0.533333   
7              26               5                 35   0.838710  0.742857   
8              34              44                 43   0.435897  0.790698   
9              28               1                 35   0.965517  0.800000   
10             15               4                 19   0.789474  0.789474   
11             11               5                 21   0.687500  0.523810   